# Notebook for creating and saving Subject objects from datasets

In [1]:
%load_ext autoreload
%autoreload 2

import os
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from subprocess import call

import sys
sys.path.append('../src/features')

from subject import Subject

cur_dir = os.getcwd()
asd_diag_dir = os.path.dirname(cur_dir)
abide_dir = os.path.dirname(asd_diag_dir) + '/abide/'
roi_200_dir = abide_dir + '/data/ABIDEI_preprocessed/Outputs/cpac/filt_noglobal/rois_cc200/'
subjects_dir = os.path.dirname(cur_dir) + '/data/ABIDEI_subjects/'
os.path.exists(roi_200_dir)

True

# Load already made subjects for modification

In [2]:
# Load ABIDEI preprocessed rois by loading subjects
def open_pickle(f):
    file = open(f,'rb')
    o = pickle.load(file)
    file.close()
    return o

def load_subjects_d(subject_folder):
    subjects_d = {}
    for f in os.listdir(subject_folder):
        s = open_pickle(os.path.join(subject_folder, f))
        subjects_d[s._sub_id] = s
    return subjects_d

subjects_d = load_subjects_d(subjects_dir)

## Subjects from ABIDE I preprocessed

In [28]:
phen_file = abide_dir + 'Phenotypic_V1_0b_preprocessed1.csv'
phen_df = pd.read_csv(phen_file)

In [36]:
subjects = list()
for index, row in phen_df.iterrows():
    data = list()
    file = row['FILE_ID'] + '_rois_cc200.1D'
    if(file in os.listdir(roi_200_dir)):
        with open(str(os.path.join(roi_200_dir, file)), newline='') as f:
            Lines = f.readlines()
            for l in Lines[1:]:
                ts = np.empty((200,))
                for n, i in enumerate(l.split("\t")):
                    ts[n] = i
                data.append(ts)       
        data_dict = {'filt_noglobal_roi_200_Cradd' : np.array(data)}
        label_dict = {'dx_group': row['DX_GROUP']}
        s = Subject(row['SUB_ID'], row['SITE_ID'], row['SEX'], row['AGE_AT_SCAN'], data_dict, label_dict)
        subjects.append(s)
assert len(subjects) == len(os.listdir(roi_200_dir))

# Add ABIDE I preprocessed

In [17]:
def open_1d_rois(file):
    data = list()
    with open(file, newline='') as f:
        Lines = f.readlines()
        for l in Lines[1:]:
            ts = np.empty((200,))
            for n, i in enumerate(l.split("\t")):
                ts[n] = i
            data.append(ts)
    return np.array(data)

def add_data(subjects_d, data_dir, data_name):
    print(f'Number of files in data dir {len(os.listdir(data_dir))}')
    for f in os.listdir(data_dir):
        sub_id = None
        for i in f.split('_'):
            if(i[0:3] == '005'):
                sub_id = int(i)
        if(sub_id is None):
            print(f'error getting sub_id from {f}')
        else:
            data = open_1d_rois(os.path.join(data_dir, f))
            s = subjects_d[sub_id]
            s._data_dict[data_name] = data
        
# data_dir = abide_dir + '/data/ABIDEI_preprocessed/Outputs/cpac/nofilt_noglobal/rois_cc200/'
# data_name = 'nofilt_noglobal_roi_200_Cradd'
# data_dir = abide_dir + '/data/ABIDEI_preprocessed/Outputs/cpac/nofilt_global/rois_cc200/'
# data_name = 'nofilt_global_roi_200_Cradd'
data_dir = abide_dir + '/data/ABIDEI_preprocessed/Outputs/cpac/filt_global/rois_cc200/'
data_name = 'filt_global_roi_200_Cradd'
add_data(subjects_d, data_dir, data_name)

Number of files in data dir 884


In [18]:
# Check that data added 
g_c = 0
for sub_id, s in subjects_d.items():
    if(data_name in s._data_dict):
        g_c += 1
print(g_c)

884


# Add CPAC preprocessed data

In [3]:
data_dir = asd_diag_dir + '/data/'
# cpac_output_dir = data_dir + 'cpac_output/output/pipeline_abide_cpac_run_freq-filter_nuisance/'
# cpac_output_dir = data_dir + 'cpac_output_cmu_a/output/pipeline_abide_cpac_run/'
# cpac_output_dir = data_dir + 'cpac_output_t/output/pipeline_abide_cpac_run/'

In [6]:
# for filter no global
# filt_noglobal = 'roi_timeseries/_scan_rest_run-1/_selector_CSF-2mmE-M_aC-CSF+WM-2mm-DPC5_M-SDB_P-2_BP-B0.01-T0.1/_mask_CC200_mask_file_..cpac_templates..CC200.nii.gz'
filt_noglobal = 'roi_timeseries/_scan_rest_run-1/_selector_CSF-2mm-M_aC-CSF+WM-2mm-DPC5_M-SDB_P-2_BP-B0.01-T0.1/_mask_CC200_mask_file_..cpac_templates..CC200.nii.gz'
def add_cpac_data(subjects_d, data_dir, data_name):
    def get_roi(sub_folder):
        roi_folder = os.path.join(sub_folder,filt_noglobal)
        if os.path.exists(os.path.join(roi_folder, 'roi_stats.npz')):
            call(['unzip', os.path.join(roi_folder, 'roi_stats.npz'), '-d', roi_folder])
            # first row is nans and skip first 5
            roi = np.load(os.path.join(roi_folder, 'arr_0.npy'), allow_pickle=True)[5:] 
        else:
            print(f"Couldn't get rois for {sub_folder}")
            roi = None
        return roi

    cpac_sub_roi_d = {}
    for sub_f in os.listdir(cpac_output_dir):
        sub_id = int(sub_f.split('-')[1].split('_')[0])
        sub_folder = os.path.join(cpac_output_dir, sub_f)
        roi = get_roi(sub_folder)
        if roi is not None:
            cpac_sub_roi_d[sub_id] = get_roi(sub_folder)
    print(f'Retrieved rois for {len(cpac_sub_roi_d)} subjects')
    for sub_id, roi in cpac_sub_roi_d.items():
        if sub_id in subjects_d:
            subjects_d[sub_id]._data_dict[data_name] = roi
        else:
            print(f'{sub_id} not in subjects dict')

In [10]:
cpac_output_dir = asd_diag_dir + '/data/aws_output/all_subjects/'
add_cpac_data(subjects_d, cpac_output_dir, 'cpac_aws_cc200')

Couldn't get rois for /home/cparish/Capstone/asd_diagnosis_fmri/data/aws_output/all_subjects/sub-0051473_ses-1
Couldn't get rois for /home/cparish/Capstone/asd_diagnosis_fmri/data/aws_output/all_subjects/sub-0051474_ses-1
Couldn't get rois for /home/cparish/Capstone/asd_diagnosis_fmri/data/aws_output/all_subjects/sub-0051171_ses-1
Couldn't get rois for /home/cparish/Capstone/asd_diagnosis_fmri/data/aws_output/all_subjects/sub-0050961_ses-1
Couldn't get rois for /home/cparish/Capstone/asd_diagnosis_fmri/data/aws_output/all_subjects/sub-0051172_ses-1
Couldn't get rois for /home/cparish/Capstone/asd_diagnosis_fmri/data/aws_output/all_subjects/sub-0050960_ses-1
Retrieved rois for 36 subjects
50952 not in subjects dict
51166 not in subjects dict
51161 not in subjects dict
50953 not in subjects dict


In [9]:
# for s in subjects:
#     if s._sub_id in cpac_sub_roi_d:
#         s._data_dict['cpac_filt_noglobal_t'] = cpac_sub_roi_d[s._sub_id]

h
h
h
h
h
h
h
h
h
h


In [12]:
# Save subjects
save_dir = os.path.dirname(cur_dir) + '/data/ABIDEI_subjects/'
for sub_id, s in subjects_d.items():
    s._save_subject(save_dir)

# Site TRs can be found on abide website or 
* https://www.researchgate.net/figure/SCANNING-PARAMETERS-OF-DIFFERENT-SITES-OF-ABIDE-1_tbl1_335722272
* https://www.nature.com/articles/sdata201710/tables/4

In [14]:
site_trs = {'CALTECH': 2, 'CMU': 2, 'KKI': '2.5', 'LEUVEN_1': 1.656, 'LEUVEN_2': 1.656, 'MAX_MUN': 3,
           'NYU': 2, 'OHSU': 2.5, 'OLIN': 2.5, 'PITT': 1.5, 'SBL': 2.2, 'SDSU': 2, 'STANFORD': 2, 
           'TRINITY': 2, 'UCLA_1': 3, 'UCLA_2': 3, 'UM_1': 2, 'UM_2': 2, 'USM': 2, 'YALE': 2}
site_trs_2 = {'BNI': 3, 'EMC': 2, 'ETH': 2, 'GU': 2, 'IU':.813, 'IP': 2.7, 'KKI_1': 2.5, 'KKI_3': 2.5, 
              'NYU_1': 2, 'NYU_2': 2, 'OHSU_1': 2.5, 'ONRC_1': .475, 'SDSU': 2, 'TCD': 2, 'UCD': 2,
             'UCLA': 3, 'USM': 2, 'UCLA_Long': 3, 'UPSM_Long': 1.5}

In [24]:
import json

trs_save_file = save_dir = os.path.dirname(cur_dir) + '/data/dicts/ABIDEI_site_trs.json'
with open(trs_save_file, 'w') as fp:
    json.dump(site_trs, fp)